In [1]:
import numpy as np
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

from pyarrow.dataset import HivePartitioning

from model.OutpatientsModel import OutpatientsModel

In [2]:
dataset = "data/synthetic"
model_run = f"test/20220110_104353"
m = OutpatientsModel(f"{dataset}/results/{model_run}")
op = pq.read_pandas(f"{dataset}/op.parquet").to_pandas()
op.drop(["hsagrp"], axis = "columns", inplace = True)

In [3]:
[sum(op.attendances), sum(op.tele_attendances), len(op.index)]

[548059, 17077, 562543]

In [4]:
selected_variant, results = m.run(0)
results

,rn,attendances,tele_attendances
0,206333,1,0
1,206184,2,0
2,206913,2,0
3,207638,1,0
4,205867,2,0
...,...,...,...
370670,5400,1,0
370671,5643,3,0
370672,7007,1,0
370673,7136,1,0


The "attendances" column shows how many outpatient attendances there were for each row of data.

In [5]:
row_count = pd.merge(
  op.value_counts("attendances").to_frame("baseline"),
  results.value_counts("attendances").to_frame("results"),
  left_index = True,
  right_index = True,
  how = "outer"
).fillna(0)

row_count.loc["total"] = [
  np.sum(row_count.baseline * row_count.index),
  np.sum(row_count.results * row_count.index)
]

row_count.astype(int)

,baseline,results
attendances,,
0,17072,16251
1,542930,198023
2,2494,104804
3,47,37871
4,0,10505
5,0,2582
6,0,512
7,0,104
8,0,21
